In [34]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [4]:
dat = pd.read_csv("./input/train_sample.csv", parse_dates=['click_time'])
dat.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0


In [5]:
dat['weekday'] = dat['click_time'].dt.weekday.astype('category', categories=range(7), ordered=True)
dat['day'] = dat['click_time'].dt.day
dat['hour'] = dat['click_time'].dt.hour
dat.head()

,ip,app,device,os,channel,click_time,attributed_time,is_attributed,weekday,day,hour
0,87540,12,1,13,497,2017-11-07 09:30:38,NaN,0,1,7,9
1,105560,25,1,17,259,2017-11-07 13:40:27,NaN,0,1,7,13
2,101424,12,1,19,212,2017-11-07 18:05:24,NaN,0,1,7,18
3,94584,13,1,13,477,2017-11-07 04:58:08,NaN,0,1,7,4
4,68413,12,1,1,178,2017-11-09 09:00:09,NaN,0,3,9,9


In [18]:
len(dat['weekday'].unique())

4

In [36]:
# split data into X and y
X = np.array(dat[['hour', 'weekday', 'day']])
#X = X.astype(str)
Y = np.array(dat['is_attributed'])

In [37]:
# encode string input values as integers
encoded_x = None
for i in range(0, X.shape[1]):
    label_encoder = LabelEncoder()
    feature = label_encoder.fit_transform(X[:,i])
    feature = feature.reshape(X.shape[0], 1)
    onehot_encoder = OneHotEncoder(sparse=False)
    feature = onehot_encoder.fit_transform(feature)
    if encoded_x is None:
        encoded_x = feature
    else:
        encoded_x = np.concatenate((encoded_x, feature), axis=1)
print("X shape: : ", encoded_x.shape)

X shape: :  (100000, 32)


In [38]:
# encode string class values as integers
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(Y)
label_encoded_y = label_encoder.transform(Y)
print("Y shape: : ", label_encoded_y.shape)

Y shape: :  (100000,)


In [39]:
X_train, X_test, y_train, y_test = train_test_split(encoded_x, label_encoded_y, test_size=0.33, random_state=7)

In [40]:
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [41]:
y_pred = model.predict(X_test)
print(y_pred)

[0 0 0 ... 0 0 0]


C:\Users\gerol\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [42]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 99.75%


In [43]:
roc = roc_auc_score(y_test, y_pred)
print("area under ROC: {}".format(roc))

area under ROC: 0.5
